In [1]:
!pip install virtualenv
!virtualenv mlops-tfx

  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
created virtual environment CPython3.10.13.final.0-64 in 687ms
  creator CPy

In [2]:
!source mlops-tfx/bin/activate

In [3]:
!pip install jupyter scikit-learn tensorflow tfx==1.14.0 flask joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of apache-beam to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of apache-beam[gcp] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-cloud-bigtable to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pydantic to determine which version is compatible with other requirements. This could

# Data Preprocessing

In [4]:
import pandas as pd
import os
df = pd.read_csv("/kaggle/input/fake-news-classification/WELFake_Dataset.csv")

In [5]:
df = df[["text", "label"]]

In [6]:
df.isnull().any()

text      True
label    False
dtype: bool

In [7]:
df = df.dropna()

In [8]:
df

,text,label
0,No comment is expected from Barack Obama Membe...,1
1,Did they post their votes for Hillary already?,1
2,"Now, most of the demonstrators gathered last ...",1
3,A dozen politically active pastors came here f...,0
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...
72129,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,"You know, because in fantasyland Republicans n...",1
72131,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [9]:
df = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 200)))

In [10]:
df['label'].value_counts()

0    200
1    200
Name: label, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 21293 to 50616
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    400 non-null    object
 1   label   400 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.4+ KB


In [12]:
df.head(2)

,text,label
21293,VATICAN CITY (Reuters) - A Vatican diplomat wo...,0
59849,"WASHINGTON, D. C. — President Donald J. Tru...",0


# Apache Beam

In [15]:
TRANSFORM_MODULE_FILE = 'fake_news_detection_transform.py'
TUNER_MODULE_FILE = 'fake_news_detection_tuner.py'
TRAINER_MODULE_FILE = 'fake_news_detection_trainer.py'
COMPONENT_MODULE_FILE = 'fake_news_detection_component.py'

In [16]:

%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import os
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

LABEL_KEY   = "label"
FEATURE_KEY = "text"

# Define the set of stopwords
stop_words = set(stopwords.words('english'))

# Renaming transformed features
def transformed_name(key):
    return key + "_xf"

# Preprocess input features into transformed features
def preprocessing_fn(inputs):
    """
    inputs:  map from feature keys to raw features
    outputs: map from feature keys to transformed features
    """

    outputs = {}

    # Convert text to lowercase
    text_lower = tf.strings.lower(inputs[FEATURE_KEY])

    # Remove stopwords
    text_without_stopwords = tf.strings.regex_replace(text_lower, '|'.join(stop_words), '')

    # Store transformed text feature
    outputs[transformed_name(FEATURE_KEY)] = text_without_stopwords

    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)


    return outputs


Writing fake_news_detection_transform.py


In [17]:


%%writefile {TUNER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY   = 'label'
FEATURE_KEY = 'text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=128) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE      = 1000
SEQUENCE_LENGTH = 500

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    embedding_dim = hp.Int('embedding_dim', min_value=30, max_value=40, step=5)
    lstm_units    = hp.Int('lstm_units', min_value=32, max_value=96, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.3, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-2])
    
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(learning_rate),
        metrics   = ['accuracy']
    )
    
    model.summary()
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 2
)

def tuner_fn(fn_args: FnArgs) -> None:
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, 5)
    val_set   = input_fn(fn_args.eval_files[0],  tf_transform_output, 5)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    
    # Build the model tuner
    model_tuner = kt.Hyperband(
        hypermodel   = lambda hp: model_builder(hp),
        objective    = kt.Objective('val_accuracy', direction='max'),
        max_epochs   = 5,
        factor       = 2,
        directory    = fn_args.working_dir,
        project_name = 'fake_news_detection'
    )
    model_tuner.oracle.max_trials = 10

    return TunerFnResult(
        tuner      = model_tuner,
        fit_kwargs = {
            'callbacks'        : [early_stop_callback],
            'x'                : train_set,
            'validation_data'  : val_set,
            'steps_per_epoch'  : fn_args.train_steps,
            'validation_steps' : fn_args.eval_steps
        }
    )
     

Writing fake_news_detection_tuner.py


In [18]:


%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY   = 'label'
FEATURE_KEY = 'text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=128) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence
VOCAB_SIZE      = 1000
SEQUENCE_LENGTH = 500
# embedding_dim   = 35

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units'], return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = ['accuracy']
    )
    
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features      = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
    
class AccuracyThresholdCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= 0.80 and logs.get('val_accuracy') >= 0.80:
            print(f"\nEpoch {epoch}: Training and validation accuracy have both reached 80%. Stopping training.")
            self.model.stop_training = True
    
def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    hp      = fn_args.hyperparameters['values']
    hp['epochs'] = 20
    
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor  = 'val_binary_accuracy',
        mode     = 'max',
        verbose  = 1,
        patience = 10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor        = 'val_binary_accuracy',
        mode           = 'max',
        verbose        = 1,
        save_best_only = True
    )
    
    accuracy_threshold_callback = AccuracyThresholdCallback()

    callbacks = [
        tensorboard_callback,
        early_stop_callback,
        model_checkpoint_callback,
        accuracy_threshold_callback
    ]
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp['tuner/epochs'])
    val_set   = input_fn(fn_args.eval_files,  tf_transform_output, hp['tuner/epochs'])

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    
    # Build the model
    model = model_builder(hp)
    
    # Train the model
    model.fit(
        x                = train_set,
        validation_data  = val_set,
        callbacks        = callbacks,
        steps_per_epoch  = fn_args.train_steps,
        validation_steps = fn_args.eval_steps,
        epochs           = hp['epochs']
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape = [None],
                dtype = tf.string,
                name  = 'examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format = 'tf',
        signatures  = signatures
    )
     

Writing fake_news_detection_trainer.py


In [19]:


%%writefile {COMPONENT_MODULE_FILE}

import os
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Tuner,
    Trainer,
    Evaluator,
    Pusher
)
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


def init_components(args):
    """Initiate tfx pipeline components

    Args:
        args (dict) :
            data_dir (str): a path to the data
            transform_module (str): a path to the transform_module
            tuner_module (str): a path to the tuner_module
            training_module (str): a path to the transform_module
            training_steps (int): number of training steps
            eval_steps (int): number of eval steps
            serving_model_dir (str): a path to the serving model directory

    Returns:
        tuple: TFX components
    """
    output = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
        ])
    )
    

    example_gen = CsvExampleGen(
        input_base=args['Data'],
        output_config=output
    )

    statistics_gen = StatisticsGen(
        examples=example_gen.outputs['examples']
    )

    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs['statistics']
    )

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )

    transform  = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(args['transform_module'])
    )
    

    tuner = Tuner(
        module_file=os.path.abspath(args['tuner_module']),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args      = trainer_pb2.TrainArgs(splits=['train']),
        eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
    )

    trainer = Trainer(
        module_file=os.path.abspath(args['trainer_module']),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        hyperparameters=tuner.outputs['best_hyperparameters'],
        train_args      = trainer_pb2.TrainArgs(splits=['train']),
        eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
    )
    

    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')

#     slicing_specs=[
#         tfma.SlicingSpec(),
#         tfma.SlicingSpec(feature_keys=[
#             'gender',
#             'ever_married'
#         ])
#     ]

    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name = 'ExampleCount'),
            tfma.MetricConfig(class_name = 'AUC'),
            tfma.MetricConfig(class_name = 'FalsePositives'),
            tfma.MetricConfig(class_name = 'TruePositives'),
            tfma.MetricConfig(class_name = 'FalseNegatives'),
            tfma.MetricConfig(class_name = 'TrueNegatives'),
            tfma.MetricConfig(class_name = 'BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold = tfma.GenericValueThreshold(
                        lower_bound = {'value': 0.5}
                    ),
                    change_threshold = tfma.GenericChangeThreshold(
                        direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute  = {'value': 0.0001}
                    )
                )
            )
        ])
    ]
    

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='label')],
        slicing_specs=[tfma.SlicingSpec()],
        metrics_specs=metrics_specs
    )

    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config
    )

    pusher = Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=args['serving_model_dir/fake_news_detection']
            )
        ),
    )
    

    return (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )

Writing fake_news_detection_component.py


# PIPELINE INITIALIZATION

In [ ]:
shutil.rmtree(destination_folder)

In [20]:
import os
import shutil

# Define the destination folder
destination_folder = '/kaggle/working/modules'  # Replace 'modules' with your desired folder name

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder)

# Define the files you want to move
files_to_move = [
    '/kaggle/working/fake_news_detection_tuner.py',
    '/kaggle/working/fake_news_detection_component.py',
    '/kaggle/working/fake_news_detection_trainer.py',
    '/kaggle/working/fake_news_detection_transform.py'
]

# Move each file to the new folder
for file_to_move in files_to_move:
    shutil.move(file_to_move, destination_folder)


In [21]:
import os
import pandas as pd
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import fake_news_detection_component


In [22]:
PIPELINE_NAME = "fake-news-detection"

# Pipeline inputs
DATA_ROOT = 'Data'
TRANSFORM_MODULE_FILE = 'modules/fake_news_detection_transform.py'
TUNER_MODULE_FILE = 'modules/fake_news_detection_tuner.py'
TRAINER_MODULE_FILE = 'modules/fake_news_detection_trainer.py'

# Pipeline outputs
OUTPUT_BASE = 'outputs'

serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, 'metadata.sqlite')

In [25]:
def init_local_pipeline(
    fake_news_detection_component, pipeline_root: Text
) -> pipeline.Pipeline:
    """Init local pipeline

    Args:
        components (dict): tfx components
        pipeline_root (Text): path to pipeline directory

    Returns:
        pipeline.Pipeline: apache beam pipeline orchestration
    """
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        '--direct_running_mode=multi_processing'
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        '----direct_num_workers=0'
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )
     

logging.set_verbosity(logging.INFO)

components = fake_news_detection_component.init_components({
    'Data': DATA_ROOT,
    'transform_module': TRANSFORM_MODULE_FILE,
    'tuner_module': TUNER_MODULE_FILE,
    'trainer_module': TRAINER_MODULE_FILE,
    'training_steps': 5000,
    'eval_steps': 1000,
    'serving_model_dir/fake_news_detection': serving_model_dir
})

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

Trial 10 Complete [00h 01m 02s]
val_accuracy: 0.7439024448394775

Best val_accuracy So Far: 0.8658536672592163
Total elapsed time: 00h 07m 28s
Results summary
Results in outputs/fake-news-detection/Tuner/.system/executor_execution/7/.temp/7/fake_news_detection
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
embedding_dim: 40
lstm_units: 96
dropout_rate: 0.3
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8658536672592163

Trial 06 summary
Hyperparameters:
embedding_dim: 40
lstm_units: 96
dropout_rate: 0.3
learning_rate: 0.01
tuner/epochs: 3
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 04
Score: 0.8414633870124817

Trial 01 summary
Hyperparameters:
embedding_dim: 40
lstm_units: 96
dropout_rate: 0.4
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8048780560493469

Trial 03 summary
Hyperparameters:
embed

In [26]:
!zip -r Project2.zip /kaggle/working/
!pip freeze > requirements.txt

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/outputs/ (stored 0%)
  adding: kaggle/working/outputs/serving_model/ (stored 0%)
  adding: kaggle/working/outputs/serving_model/1717697110/ (stored 0%)
  adding: kaggle/working/outputs/serving_model/1717697110/fingerprint.pb (stored 0%)
  adding: kaggle/working/outputs/serving_model/1717697110/variables/ (stored 0%)
  adding: kaggle/working/outputs/serving_model/1717697110/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: kaggle/working/outputs/serving_model/1717697110/variables/variables.index (deflated 66%)
  adding: kaggle/working/outputs/serving_model/1717697110/assets/ (stored 0%)
  adding: kaggle/working/outputs/serving_model/1717697110/keras_metadata.pb (deflated 90%)
  adding: kaggle/working/outputs/serving_model/1717697110/saved_model.pb (deflated 90%)
  adding: kaggle/working/outputs/fake-news-detection/ (stored 0%)
  adding: kaggle/working/outputs/fake-news-detection/metadata.sqlite (deflated 93%)
 